In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

In [ ]:
class Linear(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(5, 3)
        with torch.no_grad():
            self.linear.weight.data = torch.tensor([
                [0.2, 0.3, -0.1, 0.4, 0.2],
                [0.1, 0.2, 0.3, 0.3, 0.1],
                [0.0, 0.1, 0.2, 0.4, 0.3]
            ])
            self.linear.bias.data = torch.tensor([1.0, 2.0, 3.0])

    def forward(self, x): 
        return self.linear(x)

In [11]:
data = torch.tensor([[100.0, 102.0, 98.0, 105.0, 103.0]])

print(f"Input: {data.squeeze().numpy()}")

target = torch.tensor([[101.0, 104.0, 106.0]]) # [1, 3]
print(f"Target: {target.squeeze().numpy()}")

Input: [100. 102.  98. 105. 103.]
Target: [101. 104. 106.]


In [13]:
# Test linear
linear_model = Linear()
linear_out = linear_model.forward(data)

print("LINEAR MODEL:")
print(f" Weights:\n{linear_model.linear.weight.data.numpy()}")
print(f" Bias: {linear_model.linear.bias.data.numpy()}")
print(f" Input: {data.squeeze().numpy()}")
print(f" Output: {linear_out.squeeze().detach().numpy()}")

# Manual calculation to verify
print(" Manual calc:")
for i in range(3):
    w = linear_model.linear.weight.data[i]
    b = linear_model.linear.bias.data[i]
    result = torch.sum(w * data.squeeze()) + b
    print(f" Out{i+1}: {result:.3f}")

LINEAR MODEL:
 Weights:
[[ 0.2  0.3 -0.1  0.4  0.2]
 [ 0.1  0.2  0.3  0.3  0.1]
 [ 0.   0.1  0.2  0.4  0.3]]
 Bias: [1. 2. 3.]
 Input: [100. 102.  98. 105. 103.]
 Output: [104.399994 103.600006 105.7     ]
 Manual calc:
 Out1: 104.400
 Out2: 103.600
 Out3: 105.700


In [32]:
class DLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_trend = nn.Linear(5, 3)
        self.linear_seasonal = nn.Linear(5, 3)

        # Set weights cụ thể cho trend
        with torch.no_grad():
            self.linear_trend.weight.data = torch.tensor([
            [0.3, 0.3, 0.2, 0.1, 0.1],
            [0.2, 0.3, 0.3, 0.1, 0.1],
            [0.1, 0.2, 0.3, 0.3, 0.1]
            ])
            self.linear_trend.bias.data = torch.tensor([0.5, 1.0, 1.5])

        # Set weights cho seasonal
            self.linear_seasonal.weight.data = torch.tensor([
                [0.1, -0.2, 0.3, -0.1, 0.0],
                [0.0, 0.1, -0.2, 0.2, 0.1],
                [-0.1, 0.0, 0.1, 0.1, 0.2]])
            self.linear_seasonal.bias.data = torch.tensor([0.2, 0.3, 0.1])

    def decompose(self, x):
        # Simple moving average (window = 3) - replicate padding
        x_np = x.squeeze().numpy()
        trend = []
        L = len(x_np)
        for i in range(L):
            avg = 0.0
            if i==0:
                avg += x_np[i] + x_np[i] + x_np[i+1]
            elif i==L-1:
                avg += x_np[i-1] + x_np[i] + x_np[i]
            else:
                avg += x_np[i-1] + x_np[i] + x_np[i+1]
            avg /= 3
            trend.append(avg)

        trend_tensor = torch.tensor(trend, dtype=torch.float32).unsqueeze(0)
        seasonal = x - trend_tensor
        return trend_tensor, seasonal
    
    def forward(self, x):
        trend, seasonal = self.decompose(x)
        trend_pred = self.linear_trend(trend)
        seasonal_pred = self.linear_seasonal(seasonal)
        return trend_pred+seasonal_pred

In [33]:
# Test DLinear
dlinear_model = DLinear()
dlinear_out = dlinear_model(data)
print("DLINEAR MODEL:")
trend, seasonal = dlinear_model.decompose(data)
print(f" Input: {data.squeeze().numpy()}")
print(f" Trend: {trend.squeeze().numpy()}")
print(f" Seasonal: {seasonal.squeeze().numpy()}")
print(f" Output: {dlinear_out.squeeze().detach().numpy()}")
# Show component predictions
trend_pred = dlinear_model.linear_trend(trend)
seasonal_pred = dlinear_model.linear_seasonal(seasonal)
print(f" Trend pred: {trend_pred.squeeze().detach().numpy()}")
print(f" Seasonal pred: {seasonal_pred.squeeze().detach().numpy()}") 

DLINEAR MODEL:
 Input: [100. 102.  98. 105. 103.]
 Trend: [100.666664 100.       101.666664 102.       103.666664]
 Seasonal: [-0.6666641  2.        -3.6666641  3.        -0.6666641]
 Output: [ 99.93334 103.96666 103.     ]
 Trend pred: [101.600006 102.2      103.03333 ]
 Seasonal pred: [-1.6666658   1.7666664  -0.03333282]


In [36]:
class NLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(5, 3)
        # Set weights cụ thể
        with torch.no_grad():
            self.linear.weight.data = torch.tensor([
            [0.1, 0.2, 0.1, 0.3, 0.3],
            [0.2, 0.1, 0.3, 0.2, 0.2],
            [0.3, 0.2, 0.2, 0.2, 0.1]
            ])
            self.linear.bias.data = torch.tensor([0.5, 1.0, 2.0])
    def forward(self, x):
    # Normalize bằng last value
        last_value =x[:, -1:]
        x_norm = x - last_value
        pred_norm = self.linear(x_norm)
        pred = x + last_value
        return pred

In [37]:
# Test NLinear
nlinear_model = NLinear()
nlinear_out = nlinear_model(data)
print("NLINEAR MODEL:")
last_val = data[:, -1:]
x_norm = data - last_val
pred_norm = nlinear_model.linear(x_norm)
print(f" Input: {data.squeeze().numpy()}")
print(f" Last value: {last_val.squeeze().item()}")
print(f" Normalized: {x_norm.squeeze().numpy()}")
print(f" Pred normalized: {pred_norm.squeeze().detach().numpy()}")
print(f" Final output: {nlinear_out.squeeze().detach().numpy()}")
# Manual verification
print(" Manual calc:")
for i in range(3):
    w = nlinear_model.linear.weight.data[i]
    b = nlinear_model.linear.bias.data[i]
norm_pred = torch.sum(w * x_norm.squeeze()) + b
final = norm_pred + last_val.squeeze()
print(f" Out{i+1}: {norm_pred:.3f} + {last_val.squeeze().item():.1f} = {final:.3f}")

NLINEAR MODEL:
 Input: [100. 102.  98. 105. 103.]
 Last value: 103.0
 Normalized: [-3. -1. -5.  2.  0.]
 Pred normalized: [ 0.10000002 -0.8000001   0.29999995]
 Final output: [203. 205. 201. 208. 206.]
 Manual calc:
 Out3: 0.300 + 103.0 = 103.300
